# Caracterización de los Usuarios en Distribución

## Usuarios de la red de distribución

Los datos del consumo facturado corresponden al promedio mensual entre Junio 2024 y Junio 2025 según la NTCSD 2024.

Fuente de los datos: [Página del Coordinador](https://www.coordinador.cl/mercados/documentos/transferencias-economicas-de-empresas-distribuidoras/catastro-clientes-usuarios-en-distribucion/catastro-2025/)

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from sklearn.linear_model import LinearRegression
from pprint import pprint

## Uso del "*df_caracterizacion_residencial_incompleta*"

El "df_caracterizacion_residencial_incompleta" está hecho a partir de Caracterizacion_Dx_Jun2025_por_comuna-empresa(CEN), con principalmente, los datos del Coordinador. Considerando la gran cantidad de errores en dicho df y su posterior intento de recomposición, se le agrega el termino "_imcompleta". 

Este df para lo único que se usará será para hacer un simil entre la densidad del par Comuna-empresa y la densidad de la Comuna.

In [2]:
df_caracterizacion_residencial_incompleta = pd.read_excel("./Datos_Dx_procesados/caracterizacion_clientes_residenciales (incompleto).xlsx")

# Se elimina las columnas 'Cantidad_de_alimentadores' y 'Consumo_facturado_total_MWh', ya que serán recalculadas
df_caracterizacion_residencial_incompleta = df_caracterizacion_residencial_incompleta.drop(columns=['Cantidad_de_alimentadores',
                                                                                                    'Consumo_facturado_total_MWh'])

df_comuna_empresa = df_caracterizacion_residencial_incompleta.drop(columns=['Cantidad_de_clientes']).copy()

df_caracterizacion_residencial_incompleta

,Comuna,Distribuidor,Densidad,Mapeo_Densidad,Cantidad_de_clientes
0,algarrobo,edecsa,EXTREMADAMENTE BAJA,1,756
1,algarrobo,litoral,BAJA,3,21369
2,alhue,cge,EXTREMADAMENTE BAJA,1,3141
3,alto del carmen,cge,EXTREMADAMENTE BAJA,1,2730
4,alto hospicio,cge,MEDIA,4,36628
...,...,...,...,...,...
471,vilcun,frontel,EXTREMADAMENTE BAJA,1,6613
472,villarrica,frontel,EXTREMADAMENTE BAJA,1,0
473,villarrica,saesa,EXTREMADAMENTE BAJA,1,0
474,yumbel,frontel,MUY BAJA,2,7299


Se busca hacer un simil entre la densidad del par Comuna-Empresa (info CNE) y la densidad de la comuna en general. Para esto se realiza un promedio entre las densidades que conforman a dicha comuna ponderadas por la cantidad de clientes en dichas zonas. El resultado entonces es una tabla con la información por comuna; cantidad de clientes, alimentadores, cantidad de energía facturada, y densidad de la comuna.

In [3]:
Mapa_densidad = {
    1: "Extremadamente baja",
    2: "Muy baja",
    3: "Baja",
    4: "Media",
    5: "Alta"}

# Se suman los atributos, cantidad de clientes, energía y alimentadores por comuna (un registro por comuna)
sumas_comuna = (df_caracterizacion_residencial_incompleta.groupby("Comuna", as_index=False)
                                              .agg({"Cantidad_de_clientes": "sum"}))

# Se calcula la densidad promedio ponderada por cantidad de clientes (Mapeo_Densidad)
densidad_ponderada = (df_caracterizacion_residencial_incompleta.groupby("Comuna")[["Mapeo_Densidad", "Cantidad_de_clientes"]]
                                                    .apply(lambda g: (g["Mapeo_Densidad"] * g["Cantidad_de_clientes"]) / g["Cantidad_de_clientes"].sum())
                                                    .reset_index(name="Densidad_promedio_num"))

# Se unen sumas + densidad ponderada
df_caracterizacion_residencial_incompleta = sumas_comuna.merge(densidad_ponderada, on="Comuna", how="left")

# Se redondea al entero 1–5 el resultado del promedio y se mapea la etiqueta
df_caracterizacion_residencial_incompleta["Mapeo_densidad"] = (df_caracterizacion_residencial_incompleta["Densidad_promedio_num"].round(2).clip(1, 5).astype(int))
df_caracterizacion_residencial_incompleta["Densidad"] = df_caracterizacion_residencial_incompleta["Mapeo_densidad"].map(Mapa_densidad)
df_caracterizacion_residencial_incompleta = df_caracterizacion_residencial_incompleta.drop(columns=["Densidad_promedio_num"])

# Se eliminan columnas que no serán consideradas en el análisis
df_caracterizacion_residencial_incompleta = df_caracterizacion_residencial_incompleta.drop(columns=["Cantidad_de_clientes"])

# Se eliminan comunas que no serán consideradas en el análisis
comunas_a_eliminar = ['chepica', 'la union', 'maipu', 'san ramon']
df_caracterizacion_residencial_incompleta = df_caracterizacion_residencial_incompleta[~df_caracterizacion_residencial_incompleta["Comuna"].isin(comunas_a_eliminar)].copy()
df_caracterizacion_residencial_incompleta

,Comuna,level_1,Mapeo_densidad,Densidad
0,aisen,339,2,Muy baja
1,algarrobo,0,1,Extremadamente baja
2,algarrobo,1,2,Muy baja
3,alhue,2,1,Extremadamente baja
4,alto biobio,375,1,Extremadamente baja
...,...,...,...,...
471,yumbel,334,1,Extremadamente baja
472,yumbel,474,1,Extremadamente baja
473,yungay,335,1,Extremadamente baja
474,yungay,475,1,Extremadamente baja


## Caracterización de los clientes en distribución

Fuente de los datos: [Página web energía abierta](http://energiaabierta.cl/categorias-estadistica/electricidad/) 

Esta base de datos cuenta con la información agregada de todos los clientes **regulados** en distribución, por lo que no cuenta con los clientes libres conectados en distribución, los cuales a pesar de ser minoría, son lo que más suman energía por unidad de cliente.

Por lo tanto, para este análisis se considerarán; solo los datos de la tabla df_caracterizacion_regulados para la información sobre los clientes regulados (residenciales y no residenciales a diciembre del 2024); y solo los datos de la tabla df_caracterizacion_libres para la información de los clientes libres conectados a distribución (a Julio del 2025). Si bien hay una diferencia temporal entre ambas bases, puede ser considerada como despreciable bajo el supuesto de que entre diciembre y julio no habrían tantos 'nuevos clientes'.

Por último, en cuanto a la agregación de la demanda de los clientes libres en distribución, considerando que de aquellos de los que se tiene información son 443 de un total de 622 según los datos vistos anteriormente, se ponderará el consumo de los 443 tal que se alcancen los 0,72 TWh totales que representan a los 622 clientes libres según la información del Coordinador (df_clientes).

In [4]:
# Información a diciembre de 2024 sobre facturación de clientes regulados (CNE)
df_caracterizacion_regulados = pd.read_excel("Datos_Dx_procesados/se_facturacion_clientes_regulados (procesados).xlsx")

# Se elimina la columna Año, ya que no aporta información relevante
df_caracterizacion_regulados = df_caracterizacion_regulados.drop(columns=['Año'])

df_caracterizacion_regulados

,Region,Comuna,Tipo_clientes,Cantidad_de_clientes,Energia_promedio_mensual_2024_kWh
0,region de antofagasta,antofagasta,Residencial,147422,25692357.17
1,region de antofagasta,antofagasta,No Residencial,2655,16336378.33
2,region de antofagasta,calama,Residencial,59558,10803440.75
3,region de antofagasta,calama,No Residencial,1121,5463018.58
4,region de antofagasta,mejillones,Residencial,4696,655716.75
...,...,...,...,...,...
645,region metropolitana de santiago,talagante,No Residencial,631,3557586.75
646,region metropolitana de santiago,tiltil,Residencial,6930,1668438.83
647,region metropolitana de santiago,tiltil,No Residencial,342,2105284.92
648,region metropolitana de santiago,vitacura,Residencial,41194,12351577.00


In [5]:
# Pivotar la tabla para tener 1 fila por comuna y tipo en columnas
df_caracterizacion_regulados = (df_caracterizacion_regulados
        .pivot_table(
        index=["Region", "Comuna"],     # lo que mantiene único por fila
        columns="Tipo_clientes",        # lo que se transforma en columnas
        values=["Cantidad_de_clientes", "Energia_promedio_mensual_2024_kWh"],  # qué valores redistribuir
        aggfunc="sum",                  # cómo combinar si hay duplicados
        fill_value=0                    # opcional: rellena NaN con 0
    )
)

# Aplanar columnas MultiIndex (quedarán como 'Cantidad_de_clientes_Residencial', etc.)
df_caracterizacion_regulados.columns = [f"{col[0]}_{col[1]}" for col in df_caracterizacion_regulados.columns]
df_caracterizacion_regulados = df_caracterizacion_regulados.rename(columns={"Cantidad_de_clientes_No Residencial": "Cantidad_de_clientes_No_Residencial",
                                                                            "Energia_promedio_mensual_2024_kWh_Residencial": "Energia_promedio_mensual_Residencial_kWh",
                                                                            "Energia_promedio_mensual_2024_kWh_No Residencial": "Energia_promedio_mensual_No_Residencial_kWh"})

# Resetear índice para volver a tener Region y Comuna como columnas
df_caracterizacion_regulados = df_caracterizacion_regulados.reset_index()

# Se juntan las tablas df_caracterizacion_regulados y df_caracterizacion_regulados_incompleta
df_caracterizacion_regulados = df_caracterizacion_regulados.merge(df_caracterizacion_residencial_incompleta, on="Comuna", how="left")

# Se reordenan las columnas para la visualización
cols = [
    "Region",
    "Comuna",
    "Cantidad_de_clientes_Residencial",
    "Cantidad_de_clientes_No_Residencial",
    "Energia_promedio_mensual_Residencial_kWh",
    "Energia_promedio_mensual_No_Residencial_kWh",
    "Densidad",
    "Mapeo_densidad"
]

df_caracterizacion_regulados = df_caracterizacion_regulados[cols]
df_caracterizacion_regulados

,Region,Comuna,Cantidad_de_clientes_Residencial,Cantidad_de_clientes_No_Residencial,Energia_promedio_mensual_Residencial_kWh,Energia_promedio_mensual_No_Residencial_kWh,Densidad,Mapeo_densidad
0,region de antofagasta,antofagasta,147422,2655,25692357.17,16336378.33,Baja,3
1,region de antofagasta,antofagasta,147422,2655,25692357.17,16336378.33,Extremadamente baja,1
2,region de antofagasta,calama,59558,1121,10803440.75,5463018.58,Media,4
3,region de antofagasta,mejillones,4696,159,655716.75,1367190.25,Baja,3
4,region de antofagasta,sierra gorda,830,68,200169.17,205475.92,Extremadamente baja,1
...,...,...,...,...,...,...,...,...
466,region metropolitana de santiago,santiago,264186,10983,44114863.00,45625791.08,Alta,5
467,region metropolitana de santiago,talagante,29756,631,5573796.42,3557586.75,Baja,3
468,region metropolitana de santiago,tiltil,6930,342,1668438.83,2105284.92,Extremadamente baja,1
469,region metropolitana de santiago,tiltil,6930,342,1668438.83,2105284.92,Extremadamente baja,1


In [6]:
# Se agregan a df_caracterizacion_regulados los clientes libres de df_clientes_libres_norm_en_dx
df_caracterizacion_libres = pd.read_excel("./Datos_Dx_procesados/Caracterizacion_clientes_libres.xlsx")

df_caracterizacion_libres

,Distribuidor,Comuna,Cantidad_de_clientes_libreDx,Energia_promedio_mensual_libreDx_kWh,Cantidad_de_alimentadores
0,cec,curico,10,4203514.19,4
1,cec,romeral,6,2646471.19,2
2,cec,teno,16,8199661.19,3
3,cge,alhue,1,662063.19,2
4,cge,antofagasta,1,700931.19,25
...,...,...,...,...,...
89,luz linares,constitucion,1,990342.19,8
90,luz linares,linares,5,2789460.19,10
91,luz linares,yerbas buenas,1,624831.19,10
92,luz parral,longavi,2,1292741.19,6


In [7]:
# Se juntan las tablas df_caracterizacion_regulados, df_caracterizacion_libres y df_cantidad_alimentadores
df_caracterizacion_distribucion = df_caracterizacion_regulados.merge(df_caracterizacion_libres, on="Comuna", how="left").fillna(0)

# Se suman las cantidades totales de clientes y energía
df_caracterizacion_distribucion["Cantidad_de_clientes_libreDx"] = (df_caracterizacion_distribucion["Cantidad_de_clientes_libreDx"]).astype(int)
df_caracterizacion_distribucion["Cantidad_de_alimentadores"] = (df_caracterizacion_distribucion["Cantidad_de_alimentadores"]).astype(int)

df_caracterizacion_distribucion["Cantidad_de_clientes_Todos"] = (df_caracterizacion_distribucion["Cantidad_de_clientes_Residencial"] +
                                                             df_caracterizacion_distribucion["Cantidad_de_clientes_No_Residencial"] +
                                                             df_caracterizacion_distribucion["Cantidad_de_clientes_libreDx"])
df_caracterizacion_distribucion["Energia_promedio_mensual_Todos_kWh"] = ((df_caracterizacion_distribucion["Energia_promedio_mensual_Residencial_kWh"] +
                                                                           df_caracterizacion_distribucion["Energia_promedio_mensual_No_Residencial_kWh"] +
                                                                           df_caracterizacion_distribucion["Energia_promedio_mensual_libreDx_kWh"])).round(2)

# Se reordenan las columnas para la visualización
cols = [
    "Region",
    "Comuna",
    "Cantidad_de_clientes_Todos",
    "Cantidad_de_clientes_Residencial",
    "Cantidad_de_clientes_No_Residencial",
    "Cantidad_de_clientes_libreDx",
    "Energia_promedio_mensual_Todos_kWh",
    "Energia_promedio_mensual_Residencial_kWh",
    "Energia_promedio_mensual_No_Residencial_kWh",
    "Energia_promedio_mensual_libreDx_kWh",
    "Cantidad_de_alimentadores",
    "Densidad",
    "Mapeo_densidad"
]

df_caracterizacion_distribucion = df_caracterizacion_distribucion[cols]
df_caracterizacion_distribucion

,Region,Comuna,Cantidad_de_clientes_Todos,Cantidad_de_clientes_Residencial,Cantidad_de_clientes_No_Residencial,Cantidad_de_clientes_libreDx,Energia_promedio_mensual_Todos_kWh,Energia_promedio_mensual_Residencial_kWh,Energia_promedio_mensual_No_Residencial_kWh,Energia_promedio_mensual_libreDx_kWh,Cantidad_de_alimentadores,Densidad,Mapeo_densidad
0,region de antofagasta,antofagasta,150078,147422,2655,1,42729666.69,25692357.17,16336378.33,700931.19,25,Baja,3
1,region de antofagasta,antofagasta,150078,147422,2655,1,42729666.69,25692357.17,16336378.33,700931.19,25,Extremadamente baja,1
2,region de antofagasta,calama,60680,59558,1121,1,16906250.52,10803440.75,5463018.58,639791.19,6,Media,4
3,region de antofagasta,mejillones,4855,4696,159,0,2022907.00,655716.75,1367190.25,0.00,0,Baja,3
4,region de antofagasta,sierra gorda,898,830,68,0,405645.09,200169.17,205475.92,0.00,0,Extremadamente baja,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,region metropolitana de santiago,tiltil,7273,6930,342,1,4528934.94,1668438.83,2105284.92,755211.19,5,Extremadamente baja,1
475,region metropolitana de santiago,tiltil,7274,6930,342,2,4827050.94,1668438.83,2105284.92,1053327.19,8,Extremadamente baja,1
476,region metropolitana de santiago,tiltil,7273,6930,342,1,4528934.94,1668438.83,2105284.92,755211.19,5,Extremadamente baja,1
477,region metropolitana de santiago,tiltil,7274,6930,342,2,4827050.94,1668438.83,2105284.92,1053327.19,8,Extremadamente baja,1


In [8]:
# Entrenamiento con filas válidas (alimentadores > 0)
df_caracterizacion_distribucion_estimacion = df_caracterizacion_distribucion.copy()

df_train = df_caracterizacion_distribucion_estimacion[(df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores"] > 0)].copy()

X = df_train[["Cantidad_de_clientes_Todos", "Energia_promedio_mensual_Todos_kWh", "Mapeo_densidad"]].values
y = df_train["Cantidad_de_alimentadores"].values

# Se entrena el modelo
modelo_lineal = LinearRegression()
modelo_lineal.fit(X, y)

print("Coef clientes:", modelo_lineal.coef_[0])
print("Coef energia:", modelo_lineal.coef_[1])
print("Coef densidad:", modelo_lineal.coef_[2])
print("Constante C:", modelo_lineal.intercept_)
print("R^2 (entrenamiento):", modelo_lineal.score(X, y))

# Se crea una nueva columna sobre la cantidad de alimentadores estimada con valores nulos
df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores_estimado"] = np.nan

# Se establecen las filas a estimar (donde Cantidad_de_alimentadores == 0)
faltantes = df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores"] == 0
X_faltantes = df_caracterizacion_distribucion_estimacion.loc[faltantes, ["Cantidad_de_clientes_Todos", 
                                                                         "Energia_promedio_mensual_Todos_kWh", 
                                                                         "Mapeo_densidad"]].values

# Predicción y limpieza
y_prediccion = modelo_lineal.predict(X_faltantes)
y_prediccion = np.maximum(y_prediccion, 1)               # si se estima 0 , se asigna 1
y_prediccion = np.round(y_prediccion).astype(int)        # Deben ser enteros

# Se guarda en la nueva columna
df_caracterizacion_distribucion_estimacion.loc[faltantes, "Cantidad_de_alimentadores_estimado"] = y_prediccion
df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores_estimado"] = (df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores_estimado"]
                                                                                    .fillna(0).astype(int))

Coef clientes: 0.00011051747846068749
Coef energia: 7.030117476486491e-08
Coef densidad: 3.172430471238378
Constante C: -0.7925581487547753
R^2 (entrenamiento): 0.6209250599991332


In [9]:
# Se actualiza la columna 'Cantidad_de_alimentadores' en el df_caracterizacion_distribucion
df_caracterizacion_distribucion["Cantidad_de_alimentadores"] = (df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores"]
                                                                    .where(df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores"] > 0, 
                                                                            df_caracterizacion_distribucion_estimacion["Cantidad_de_alimentadores_estimado"])).astype(int)

df_caracterizacion_distribucion = df_caracterizacion_distribucion.drop(columns=["Mapeo_densidad"])
df_caracterizacion_distribucion

,Region,Comuna,Cantidad_de_clientes_Todos,Cantidad_de_clientes_Residencial,Cantidad_de_clientes_No_Residencial,Cantidad_de_clientes_libreDx,Energia_promedio_mensual_Todos_kWh,Energia_promedio_mensual_Residencial_kWh,Energia_promedio_mensual_No_Residencial_kWh,Energia_promedio_mensual_libreDx_kWh,Cantidad_de_alimentadores,Densidad
0,region de antofagasta,antofagasta,150078,147422,2655,1,42729666.69,25692357.17,16336378.33,700931.19,25,Baja
1,region de antofagasta,antofagasta,150078,147422,2655,1,42729666.69,25692357.17,16336378.33,700931.19,25,Extremadamente baja
2,region de antofagasta,calama,60680,59558,1121,1,16906250.52,10803440.75,5463018.58,639791.19,6,Media
3,region de antofagasta,mejillones,4855,4696,159,0,2022907.00,655716.75,1367190.25,0.00,9,Baja
4,region de antofagasta,sierra gorda,898,830,68,0,405645.09,200169.17,205475.92,0.00,3,Extremadamente baja
...,...,...,...,...,...,...,...,...,...,...,...,...
474,region metropolitana de santiago,tiltil,7273,6930,342,1,4528934.94,1668438.83,2105284.92,755211.19,5,Extremadamente baja
475,region metropolitana de santiago,tiltil,7274,6930,342,2,4827050.94,1668438.83,2105284.92,1053327.19,8,Extremadamente baja
476,region metropolitana de santiago,tiltil,7273,6930,342,1,4528934.94,1668438.83,2105284.92,755211.19,5,Extremadamente baja
477,region metropolitana de santiago,tiltil,7274,6930,342,2,4827050.94,1668438.83,2105284.92,1053327.19,8,Extremadamente baja


In [10]:
# Información a junio 2025
# -> Cantidad de clientes
# -> Cantidad de energía
cantidad_clientes_en_distribucion = df_caracterizacion_distribucion['Cantidad_de_clientes_Todos'].sum()
cantidad_energia_en_distribucion = df_caracterizacion_distribucion['Energia_promedio_mensual_Todos_kWh'].sum()

print("(A fecha de diciembre 2024)",
      "\nClientes en distribución:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_en_distribucion),
      "\n -> Energía facturada mensual: {:,} TWh".format((cantidad_energia_en_distribucion / 1_000_000_000).round(2)))

(A fecha de diciembre 2024) 
Clientes en distribución: 
 -> Cantidad de clientes: 11,675,029 
 -> Energía facturada mensual: 4.23 TWh


In [11]:
# Se reestructura el df_caracterizacion_distribucion para tener una fila por comuna y segmento
id_cols = ["Region", "Comuna", "Cantidad_de_alimentadores", "Densidad"]

# Se crean dos dataframes "derretidos" y luego se unen
df_cantidad_de_clientes = df_caracterizacion_distribucion.melt(
    id_vars=id_cols,
    value_vars=[
        "Cantidad_de_clientes_Residencial",
        "Cantidad_de_clientes_No_Residencial",
        "Cantidad_de_clientes_libreDx",
        "Cantidad_de_clientes_Todos"
    ],
    var_name="tmp", value_name="Cantidad_de_clientes"
)

# Segmento correcto para clientes: quitar el prefijo
df_cantidad_de_clientes["Segmento"] = df_cantidad_de_clientes["tmp"].str.replace(
    r"^Cantidad_de_clientes_", "", regex=True
)

# Se crea el segundo dataframe "derretido"
df_energia_promedio_mensual = df_caracterizacion_distribucion.melt(
    id_vars=id_cols,
    value_vars=[
        "Energia_promedio_mensual_Residencial_kWh",
        "Energia_promedio_mensual_No_Residencial_kWh",
        "Energia_promedio_mensual_libreDx_kWh",
        "Energia_promedio_mensual_Todos_kWh"
    ],
    var_name="tmp", value_name="Energia_promedio_mensual_kWh"
)

# Segmento correcto para energía: quitar prefijo y el sufijo _kWh
df_energia_promedio_mensual["Segmento"] = (df_energia_promedio_mensual["tmp"]
    .str.replace(r"^Energia_promedio_mensual_", "", regex=True)
    .str.replace(r"_kWh$", "", regex=True)
)

# Se unen usando TODAS las columnas identificadoras + Segmento
df_caracterizacion_distribucion = (df_cantidad_de_clientes[id_cols + ["Segmento", "Cantidad_de_clientes"]]
       .merge(df_energia_promedio_mensual[id_cols + ["Segmento", "Energia_promedio_mensual_kWh"]],
              on=id_cols + ["Segmento"], how="inner")
       .sort_values(id_cols)
       .reset_index(drop=True))

# Se reordenan las columnas para la visualización
cols = [
    "Region",
    "Comuna",
    "Segmento",
    "Cantidad_de_clientes",
    "Energia_promedio_mensual_kWh",
    "Cantidad_de_alimentadores",
    "Densidad"
]

df_caracterizacion_distribucion = df_caracterizacion_distribucion[cols]
df_caracterizacion_distribucion

,Region,Comuna,Segmento,Cantidad_de_clientes,Energia_promedio_mensual_kWh,Cantidad_de_alimentadores,Densidad
0,region de antofagasta,antofagasta,Residencial,147422,25692357.17,25,Baja
1,region de antofagasta,antofagasta,No_Residencial,2655,16336378.33,25,Baja
2,region de antofagasta,antofagasta,libreDx,1,700931.19,25,Baja
3,region de antofagasta,antofagasta,Todos,150078,42729666.69,25,Baja
4,region de antofagasta,antofagasta,Residencial,147422,25692357.17,25,Extremadamente baja
...,...,...,...,...,...,...,...
3167,region metropolitana de santiago,tiltil,Todos,7274,4827050.94,8,Extremadamente baja
3168,region metropolitana de santiago,vitacura,Residencial,41194,12351577.00,29,Media
3169,region metropolitana de santiago,vitacura,No_Residencial,3208,13597615.83,29,Media
3170,region metropolitana de santiago,vitacura,libreDx,4,5630897.19,29,Media


In [ ]:
# Información adicional a completar en el futuro
df_caracterizacion_distribucion["Perdidas_kWh"] = np.nan
df_caracterizacion_distribucion["Cantidad_de_fallas_mensual"] = np.nan
df_caracterizacion_distribucion["Tipo_de_medicion"] = np.nan

# Se renombran las columnas para su correcta interpretación
df_caracterizacion_distribucion_export = df_caracterizacion_distribucion.rename(columns={
    "Cantidad_de_clientes": "Cantidad de clientes",
    "Energia_promedio_mensual_kWh": "Energía promedio mensual [kWh]",
    "Cantidad_de_alimentadores": "Cantidad de alimentadores",
    "Densidad": "Densidad",
    "Perdidas_kWh": "Pérdidas [kWh]",
    "Cantidad_de_fallas_mensual": "Cantidad de fallas mensual",
    "Tipo_de_medicion": "Tipo de medición"
})

df_caracterizacion_distribucion_export.to_excel("./Salidas/Caracterizacion_Dx_Dic2024_comuna (EA).xlsx", index=True)